In [13]:
!pip install AugmentedSocialScientist pandas numpy

In [14]:
import os
# os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

from torch import cuda

cuda.get_device_name(0)

'Tesla T4'

In [15]:
import pandas as pd

annotations_file = pd.read_csv("../data/intermediate/current_annotation_file.csv")
annotations_file.sample(5)

,sentence_id,Personne,sentence_text
1027,1230132,NaN,Ces t ravaux conduisent à estimer que l’ inci...
682,1280766,Personne à protéger,Le résultat de cette inadaptation temporelle e...
326,900430,Genre/espèce humaine,On \ntouche ici en effet au plus secret de l’i...
672,680029,Individu,"par mois. Une telle somme, qui équivaut à pein..."
108,1280975,Personne à protéger,- elle produit l’amélioration de la planificat...


In [16]:
annotations = annotations_file.dropna()
annotations.sample(5)

,sentence_id,Personne,sentence_text
148,410043,Individu,- La protection des personnes\nLe bénéfice pro...
356,450540,Individu,Ce que masque ainsi l'argument - d'ailleurs no...
498,1210935,Personne à protéger,Le Comité ne semble \ndonc reconnaître aucu...
540,1280889,Agent responsable,Pour se convaincre de l’autonom ie très \nrela...
52,380138,Agent responsable,"photos, films) ont été recueillies, les person..."


In [17]:
annotations["Personne"].value_counts()

Personne
Personne à protéger       143
Matériel biologique       127
Individu                  118
Agent responsable         104
Personne relationnelle     97
Genre/espèce humaine       90
Humain comme empathie      10
Poubelle                    8
Name: count, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split
 
train_data, test_data = train_test_split(annotations, train_size=0.67)

In [19]:
from AugmentedSocialScientist.models import Camembert

bert = Camembert()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


There are 1 GPU(s) available.
We will use GPU 0: Tesla T4


In [20]:
batch_size = 32
train_loader = bert.encode(train_data.sentence_text.values,
train_data.Personne.values, batch_size=batch_size)

  0%|          | 0/466 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1022 > 512). Running this sequence through the model will result in indexing errors


  0%|          | 0/466 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [21]:
test_loader = bert.encode(test_data.sentence_text.values,
test_data.Personne.values, batch_size=batch_size)

  0%|          | 0/231 [00:00<?, ?it/s]

  0%|          | 0/231 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}


In [22]:
scores = bert.run_training(
    train_loader,
    test_loader,
    n_epochs=10,
    lr=1e-5,
    random_state=1968,
    save_model_as='clickbait'
    )

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/mamba/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



======== Epoch 1 / 10 ========
Training...


OutOfMemoryError: CUDA out of memory. Tried to allocate 384.00 MiB. GPU 0 has a total capacity of 14.58 GiB of which 329.56 MiB is free. Process 2585209 has 14.25 GiB memory in use. Of the allocated memory 14.08 GiB is allocated by PyTorch, and 40.80 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
help(bert.run_training)

Help on method run_training in module AugmentedSocialScientist.bert_base:

run_training(train_dataloader: torch.utils.data.dataloader.DataLoader, test_dataloader: torch.utils.data.dataloader.DataLoader, n_epochs: int = 3, lr: float = 5e-05, random_state: int = 42, save_model_as: str | None = None) -> Tuple[Any, Any, Any, Any] method of AugmentedSocialScientist.models.Camembert instance
    Train, evaluate and save a BERT model.
    
    Parameters
    ----------
    train_dataloader: torch.Dataloader 
        training dataloader obtained with self.encode()
    
    test_dataloader: torch.Dataloader 
        test dataloader obtained with self.encode()
    
    n_epochs: int, default=3
        number of epochs
    
    lr: float, default=5e-5
        learning rate
    
    random_state: int, default=42
        random state (for replicability)
    
    save_model_as: str, default=None
        name of model to save as. The model will be saved at ./models/<model_name>. If None, not saving t

In [ ]:
import numpy as np
pred_data = annotations_file[annotations_file.isna().any(axis=1)].drop('Personne', axis=1).dropna()
pred_loader = bert.encode(pred_data.sentence_text.values, batch_size=batch_size)
pred_proba = bert.predict_with_model(pred_loader, model_path='./models/clickbait')
pred_data['pred_label'] = np.argmax(pred_proba, axis=1)
pred_data['pred_proba'] = np.max(pred_proba, axis=1)

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/798 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

label ids: {'Agent responsable': 0, 'Genre/espèce humaine': 1, 'Humain comme empathie': 2, 'Individu': 3, 'Matériel biologique': 4, 'Personne relationnelle': 5, 'Personne à protéger': 6, 'Poubelle': 7}
